In [1]:
import sys
sys.path.append('..')
import config
from fqc import uccsd, util

import numpy as np
from datetime import datetime

data_path = config.DATA_PATH
file_name = datetime.today().strftime('%h%d')

In [2]:
from quantum_optimal_control.helper_functions.grape_functions import transmon_gate
from quantum_optimal_control.main_grape.grape import Grape
from quantum_optimal_control.core import hamiltonian

In [3]:
d = 2  # this is the number of energy levels to consider (i.e. d-level qudits)
max_iterations = 2000
decay =  max_iterations / 2
convergence = {'rate':0.05, 'max_iterations': max_iterations,
               'conv_target':1e-3, 'learning_rate_decay':decay, 'min_grad': 1e-12, 'update_step': 20}
reg_coeffs = {}

In [4]:
def binary_search_for_shortest_pulse_time(min_time, max_time):
    """Search between [min_time, max_time] up to 1ns tolerance. Assumes 20 steps per ns."""
    min_steps, max_steps = min_time * 20, max_time * 20
    while min_steps + 20 < max_steps:  # just estimate to +- 1ns
        mid_steps = int((min_steps + max_steps) / 2)
        total_time = mid_steps / 20.0
        print('\n\ntrying total_time: %s for unitary of size %s' % (str(total_time), str(U.shape)))
        SS = Grape(H0, Hops, Hnames, U, total_time, mid_steps, states_concerned_list, convergence,
                         reg_coeffs=reg_coeffs,
                         use_gpu=False, sparse_H=False, method='L-BFGS-B', maxA=maxA,
                         show_plots=False, file_name=file_name, data_path=data_path)
        if SS.l < SS.conv.conv_target:  # if converged, search lower half
            max_steps = mid_steps
        else:
            min_steps = mid_steps

    return mid_steps / 20

----
## H2 (35.3 ns w/ gate based compilation)

In [17]:
circuit = uccsd.get_uccsd_circuit('H2')
U = util.circuitutil.get_unitary(circuit)

N = 2
connected_qubit_pairs = util.get_nearest_neighbor_coupling_list(1, 2, directed=False)
H0 = hamiltonian.get_H0(N, d)
Hops, Hnames = hamiltonian.get_Hops_and_Hnames(N, d, connected_qubit_pairs)
states_concerned_list = hamiltonian.get_full_states_concerned_list(N, d)
maxA = hamiltonian.get_maxA(N, d, connected_qubit_pairs)

In [21]:
shortest_time = binary_search_for_shortest_pulse_time(0, 35)



trying total_time: 17.5 for unitary of size (4, 4)
data saved at: /project/ftchong/qoc/pranav/00068_Mar10.h5
Using 4 Taylor terms and 3 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Starting L-BFGS-B Optimization
Error = :9.86e-01; Runtime: 0.5s; Iterations = 0, grads =   3.288e-03, unitary_metric = 1.00003
Error = :4.11e-01; Runtime: 3.5s; Iterations = 20, grads =   3.169e+00, unitary_metric = 0.99999
Error = :2.31e-02; Runtime: 6.1s; Iterations = 40, grads =   1.915e-02, unitary_metric = 0.99999
Error = :5.72e-03; Runtime: 8.7s; Iterations = 60, grads =   4.694e-03, unitary_metric = 1.00000
Target fidelity reached
L-BFGS-B optimization done
b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
Error = 8.17e-04
Total time is 11.303666

Error = 8.94e-04
Total time is 4.168631315231323
Error = :8.94e-04; Runtime: 4.2s; Iterations = 108, grads =   3.413e-05, unitary_metric = 1.00000
data saved at: /project/ftchong/qoc/pranav/00073_Mar10.h5


In [23]:
print(shortest_time)

2.7


----
## LiH (871.7 ns w/ gate based compilation)

In [11]:
circuit = uccsd.get_uccsd_circuit('LiH')
U = util.circuitutil.get_unitary(circuit)

N = 4
connected_qubit_pairs = util.get_nearest_neighbor_coupling_list(2, 2, directed=False)
H0 = np.zeros((d ** N, d ** N))
Hops, Hnames = hamiltonian.get_Hops_and_Hnames(N, d, connected_qubit_pairs)
states_concerned_list = hamiltonian.get_full_states_concerned_list(N, d)
maxA = hamiltonian.get_maxA(N, d, connected_qubit_pairs)

In [12]:
shortest_time = binary_search_for_shortest_pulse_time(0, 200)



trying total_time: 100.0 for unitary of size (16, 16)
data saved at: /project/ftchong/qoc/pranav/00087_Mar10.h5
Using 8 Taylor terms and 2 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Starting L-BFGS-B Optimization
Error = :9.99e-01; Runtime: 3.5s; Iterations = 0, grads =   7.573e-04, unitary_metric = 1.00018
Error = :9.77e-01; Runtime: 27.8s; Iterations = 20, grads =   8.379e-02, unitary_metric = 1.00012
Error = :6.77e-01; Runtime: 48.8s; Iterations = 40, grads =   6.825e-02, unitary_metric = 1.00010
Error = :6.15e-01; Runtime: 67.9s; Iterations = 60, grads =   3.129e-02, unitary_metric = 1.00009
Error = :5.83e-01; Runtime: 88.0s; Iterations = 80, grads =   1.553e-02, unitary_metric = 1.00009
Error = :5.58e-01; Runtime: 105.5s; Ite

Error = :4.22e-01; Runtime: 95.8s; Iterations = 200, grads =   2.603e-02, unitary_metric = 1.00027
Error = :3.86e-01; Runtime: 105.8s; Iterations = 220, grads =   8.378e-03, unitary_metric = 1.00026
Error = :3.60e-01; Runtime: 114.6s; Iterations = 240, grads =   1.766e-02, unitary_metric = 1.00019
Error = :3.36e-01; Runtime: 124.1s; Iterations = 260, grads =   2.082e-02, unitary_metric = 1.00019
Error = :3.05e-01; Runtime: 132.9s; Iterations = 280, grads =   4.439e-02, unitary_metric = 1.00013
Error = :2.79e-01; Runtime: 142.3s; Iterations = 300, grads =   4.065e-03, unitary_metric = 1.00009
Error = :2.57e-01; Runtime: 152.6s; Iterations = 320, grads =   7.465e-03, unitary_metric = 1.00013
Error = :2.39e-01; Runtime: 161.8s; Iterations = 340, grads =   1.119e-02, unitary_metric = 1.00014
Error = :2.24e-01; Runtime: 170.8s; Iterations = 360, grads =   4.991e-03, unitary_metric = 1.00009
Error = :2.11e-01; Runtime: 179.8s; Iterations = 380, grads =   5.177e-03, unitary_metric = 1.00007
E

Error = :5.31e-02; Runtime: 148.9s; Iterations = 640, grads =   1.156e-03, unitary_metric = 0.99994
Error = :4.59e-02; Runtime: 153.5s; Iterations = 660, grads =   1.876e-03, unitary_metric = 0.99993
Error = :4.03e-02; Runtime: 157.9s; Iterations = 680, grads =   3.394e-04, unitary_metric = 0.99993
Error = :3.58e-02; Runtime: 162.3s; Iterations = 700, grads =   5.678e-04, unitary_metric = 0.99994
Error = :3.28e-02; Runtime: 167.4s; Iterations = 720, grads =   3.993e-04, unitary_metric = 0.99994
Error = :3.05e-02; Runtime: 172.0s; Iterations = 740, grads =   3.503e-04, unitary_metric = 0.99993
Error = :2.91e-02; Runtime: 176.5s; Iterations = 760, grads =   1.793e-02, unitary_metric = 0.99993
Error = :2.55e-02; Runtime: 181.1s; Iterations = 780, grads =   8.797e-04, unitary_metric = 0.99993
Error = :2.31e-02; Runtime: 185.7s; Iterations = 800, grads =   3.283e-04, unitary_metric = 0.99993
Error = :2.12e-02; Runtime: 190.8s; Iterations = 820, grads =   3.800e-03, unitary_metric = 0.99993


Error = :1.41e-01; Runtime: 106.6s; Iterations = 760, grads =   1.888e-04, unitary_metric = 0.99998
Error = :1.37e-01; Runtime: 109.6s; Iterations = 780, grads =   6.212e-04, unitary_metric = 0.99999
Error = :1.36e-01; Runtime: 112.6s; Iterations = 800, grads =   8.053e-05, unitary_metric = 0.99998
Error = :1.33e-01; Runtime: 115.9s; Iterations = 820, grads =   4.449e-04, unitary_metric = 0.99999
Error = :1.30e-01; Runtime: 118.9s; Iterations = 840, grads =   6.596e-05, unitary_metric = 0.99998
Error = :1.27e-01; Runtime: 121.9s; Iterations = 860, grads =   1.315e-04, unitary_metric = 0.99998
Error = :1.25e-01; Runtime: 125.0s; Iterations = 880, grads =   7.099e-04, unitary_metric = 0.99999
Error = :1.23e-01; Runtime: 128.0s; Iterations = 900, grads =   1.360e-04, unitary_metric = 0.99998
Error = :1.21e-01; Runtime: 131.2s; Iterations = 920, grads =   1.658e-04, unitary_metric = 0.99998
Error = :1.20e-01; Runtime: 133.9s; Iterations = 940, grads =   6.096e-05, unitary_metric = 0.99999


Error = :9.95e-02; Runtime: 104.3s; Iterations = 540, grads =   6.148e-04, unitary_metric = 0.99996
Error = :8.80e-02; Runtime: 108.2s; Iterations = 560, grads =   5.529e-04, unitary_metric = 0.99996
Error = :8.11e-02; Runtime: 112.2s; Iterations = 580, grads =   5.642e-04, unitary_metric = 0.99996
Error = :7.26e-02; Runtime: 116.2s; Iterations = 600, grads =   6.265e-04, unitary_metric = 0.99996
Error = :6.66e-02; Runtime: 120.5s; Iterations = 620, grads =   1.635e-03, unitary_metric = 0.99996
Error = :6.00e-02; Runtime: 124.5s; Iterations = 640, grads =   6.290e-04, unitary_metric = 0.99996
Error = :5.33e-02; Runtime: 128.5s; Iterations = 660, grads =   4.252e-04, unitary_metric = 0.99996
Error = :4.84e-02; Runtime: 132.5s; Iterations = 680, grads =   4.103e-04, unitary_metric = 0.99997
Error = :4.46e-02; Runtime: 136.5s; Iterations = 700, grads =   2.847e-04, unitary_metric = 0.99997
Error = :4.12e-02; Runtime: 140.8s; Iterations = 720, grads =   1.865e-04, unitary_metric = 0.99997


Error = :3.17e-01; Runtime: 66.4s; Iterations = 320, grads =   1.106e-03, unitary_metric = 0.99996
Error = :2.96e-01; Runtime: 70.5s; Iterations = 340, grads =   1.789e-03, unitary_metric = 0.99995
Error = :2.69e-01; Runtime: 74.4s; Iterations = 360, grads =   1.636e-03, unitary_metric = 0.99995
Error = :2.46e-01; Runtime: 78.3s; Iterations = 380, grads =   7.489e-03, unitary_metric = 0.99996
Error = :2.27e-01; Runtime: 82.2s; Iterations = 400, grads =   2.086e-03, unitary_metric = 0.99995
Error = :2.13e-01; Runtime: 87.1s; Iterations = 420, grads =   1.116e-02, unitary_metric = 0.99995
Error = :2.02e-01; Runtime: 91.4s; Iterations = 440, grads =   1.059e-03, unitary_metric = 0.99994
Error = :1.89e-01; Runtime: 95.6s; Iterations = 460, grads =   7.912e-04, unitary_metric = 0.99995
Error = :1.77e-01; Runtime: 99.7s; Iterations = 480, grads =   2.510e-03, unitary_metric = 0.99995
Error = :1.64e-01; Runtime: 103.6s; Iterations = 500, grads =   1.145e-03, unitary_metric = 0.99995
Error = :

Error = :5.66e-01; Runtime: 21.4s; Iterations = 100, grads =   3.386e-03, unitary_metric = 0.99998
Error = :5.22e-01; Runtime: 25.8s; Iterations = 120, grads =   4.640e-03, unitary_metric = 0.99998
Error = :4.40e-01; Runtime: 29.9s; Iterations = 140, grads =   7.126e-03, unitary_metric = 0.99997
Error = :4.08e-01; Runtime: 33.9s; Iterations = 160, grads =   4.643e-03, unitary_metric = 0.99996
Error = :3.93e-01; Runtime: 37.8s; Iterations = 180, grads =   2.695e-03, unitary_metric = 0.99997
Error = :3.86e-01; Runtime: 41.9s; Iterations = 200, grads =   9.897e-04, unitary_metric = 0.99996
Error = :3.77e-01; Runtime: 46.5s; Iterations = 220, grads =   8.517e-04, unitary_metric = 0.99997
Error = :3.66e-01; Runtime: 50.7s; Iterations = 240, grads =   1.424e-03, unitary_metric = 0.99996
Error = :3.52e-01; Runtime: 54.7s; Iterations = 260, grads =   4.346e-03, unitary_metric = 0.99996
Error = :3.35e-01; Runtime: 58.9s; Iterations = 280, grads =   1.785e-03, unitary_metric = 0.99997
Error = :3

Error = :6.42e-01; Runtime: 13.4s; Iterations = 60, grads =   5.542e-03, unitary_metric = 1.00000
Error = :6.00e-01; Runtime: 17.1s; Iterations = 80, grads =   2.235e-03, unitary_metric = 1.00000
Error = :5.87e-01; Runtime: 21.2s; Iterations = 100, grads =   8.138e-03, unitary_metric = 0.99999
Error = :5.71e-01; Runtime: 25.3s; Iterations = 120, grads =   5.193e-03, unitary_metric = 0.99999
Error = :5.36e-01; Runtime: 28.9s; Iterations = 140, grads =   5.249e-03, unitary_metric = 0.99998
Error = :5.08e-01; Runtime: 32.8s; Iterations = 160, grads =   4.647e-03, unitary_metric = 0.99998
Error = :4.67e-01; Runtime: 36.8s; Iterations = 180, grads =   1.116e-02, unitary_metric = 0.99997
Error = :4.23e-01; Runtime: 40.8s; Iterations = 200, grads =   4.746e-03, unitary_metric = 0.99997
Error = :3.57e-01; Runtime: 45.1s; Iterations = 220, grads =   6.869e-03, unitary_metric = 0.99997
Error = :3.32e-01; Runtime: 49.1s; Iterations = 240, grads =   8.039e-03, unitary_metric = 0.99997
Error = :3.1

In [13]:
print(shortest_time)

22.6


----
## BeH2 (5308.3 ns w/ gate based compilation)

In [5]:
circuit = uccsd.get_uccsd_circuit('BeH2')
U = util.circuitutil.get_unitary(circuit)

N = 6
connected_qubit_pairs = util.get_nearest_neighbor_coupling_list(2, 3, directed=False)
H0 = np.zeros((d ** N, d ** N))
Hops, Hnames = hamiltonian.get_Hops_and_Hnames(N, d, connected_qubit_pairs)
states_concerned_list = hamiltonian.get_full_states_concerned_list(N, d)
maxA = hamiltonian.get_maxA(N, d, connected_qubit_pairs)

In [6]:
d = 2  # this is the number of energy levels to consider (i.e. d-level qudits)
max_iterations = 3000
decay =  max_iterations / 2
convergence = {'rate':0.05, 'max_iterations': max_iterations,
               'conv_target':1e-3, 'learning_rate_decay':decay, 'min_grad': 1e-12, 'update_step': 20}
reg_coeffs = {}

In [ ]:
shortest_time = binary_search_for_shortest_pulse_time(100, 700)



trying total_time: 400.0 for unitary of size (64, 64)
data saved at: /project/ftchong/qoc/pranav/00002_Mar11.h5
Using 8 Taylor terms and 3 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Starting L-BFGS-B Optimization
Error = :1.00e+00; Runtime: 24.1s; Iterations = 0, grads =   1.091e-04, unitary_metric = 1.00208
Error = :9.97e-01; Runtime: 276.7s; Iterations = 20, grads =   4.776e-03, unitary_metric = 1.00181
Error = :9.76e-01; Runtime: 517.1s; Iterations = 40, grads =   3.600e-03, unitary_metric = 1.00189
Error = :9.68e-01; Runtime: 743.3s; Iterations = 60, grads =   1.608e-03, unitary_metric = 1.00185
Error = :9.60e-01; Runtime: 978.1s; Iterations = 80, grads =   4.222e-03, unitary_metric = 1.00189
Error = :9.55e-01; Runtime: 1202.8

Error = :5.80e-01; Runtime: 17219.6s; Iterations = 1520, grads =   6.363e-03, unitary_metric = 1.00242
Error = :5.76e-01; Runtime: 17438.3s; Iterations = 1540, grads =   2.833e-03, unitary_metric = 1.00236
Error = :5.73e-01; Runtime: 17659.5s; Iterations = 1560, grads =   2.161e-03, unitary_metric = 1.00233
Error = :5.70e-01; Runtime: 17881.1s; Iterations = 1580, grads =   1.411e-02, unitary_metric = 1.00231
Error = :5.67e-01; Runtime: 18102.7s; Iterations = 1600, grads =   6.287e-03, unitary_metric = 1.00225
Error = :5.64e-01; Runtime: 18342.4s; Iterations = 1620, grads =   6.575e-03, unitary_metric = 1.00218
Error = :5.61e-01; Runtime: 18568.3s; Iterations = 1640, grads =   8.509e-03, unitary_metric = 1.00213
Error = :5.58e-01; Runtime: 18788.7s; Iterations = 1660, grads =   1.611e-03, unitary_metric = 1.00212
Error = :5.55e-01; Runtime: 19009.5s; Iterations = 1680, grads =   2.726e-03, unitary_metric = 1.00212
Error = :5.53e-01; Runtime: 19232.1s; Iterations = 1700, grads =   3.397e

Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Starting L-BFGS-B Optimization
Error = :1.00e+00; Runtime: 30.2s; Iterations = 0, grads =   4.088e-05, unitary_metric = 1.00184
Error = :9.99e-01; Runtime: 389.0s; Iterations = 20, grads =   4.482e-03, unitary_metric = 1.00239
Error = :9.81e-01; Runtime: 726.4s; Iterations = 40, grads =   7.752e-04, unitary_metric = 1.00239
Error = :9.75e-01; Runtime: 1069.6s; Iterations = 60, grads =   1.751e-03, unitary_metric = 1.00226
Error = :9.71e-01; Runtime: 1406.7s; Iterations = 80, grads =   1.554e-03, unitary_metric = 1.00218
Error = :9.65e-01; Runtime: 1745.8s; Iterations = 100, grads =   2.357e-03, unitary_metric = 1.00234
Error = :9.50e-01; Runtime: 2107.9s; Iterations = 120, grads =   9.362e-03, unitary_metric = 1.00269
Error = :9.1

In [ ]:
print(shortest_time)

----
## NaH (5490.4 ns w/ gate based compilation)

----
## H2O (33842.2 ns w/ gate based compilation)